In [6]:
#Module imports
#!python -m pip install geeViz --upgrade
try:
    import geeViz.getImagesLib as getImagesLib
except:
    !python -m pip install geeViz
    import geeViz.getImagesLib as getImagesLib

import geeViz.changeDetectionLib as changeDetectionLib
import geeViz.assetManagerLib as aml
import geeViz.taskManagerLib as tml
import geeViz.gee2Pandas as g2p

import inspect,os

ee = getImagesLib.ee
Map = getImagesLib.Map

print('Done')

Done


In [4]:
pre_baked_path_root  = 'projects/disco-glass-413820/assets/users'
export_path_root = pre_baked_path_root

In [5]:
 #Create folder and a collection and make public

export_composite_collection = f'{export_path_root}/composites2'
export_landTrendr_collection = f'{export_path_root}/LT0220'

aml.create_asset(export_landTrendr_collection,asset_type = ee.data.ASSET_TYPE_IMAGE_COLL)

# Currently geeView within Colab uses a different project to authenticate through, so you may need to make your asset public to view from within Colab
aml.updateACL(export_landTrendr_collection,writers = [],all_users_can_read = True,readers = [])

print('Done')

Found the following sub directories:  ['users', 'LT0220']
Will attempt to create them if they do not exist
Asset projects/disco-glass-413820/assets/users already exists
projects/disco-glass-413820/assets/users/LT0220
New asset projects/disco-glass-413820/assets/users/LT0220 created
Updating permissions for:  projects/disco-glass-413820/assets/users/LT0220
Done


In [11]:
# Bring in composites and pull info from them
composites = ee.ImageCollection(export_composite_collection) #mine
props = composites.first().toDictionary().getInfo()

startYear = props['startYear']
endYear = props['endYear']

startJulian = props['startJulian']
endJulian = props['endJulian']

proj = composites.first().projection().getInfo()

# Pull out the crs
# Depending on if a wkt or epsg format is used, it will be stored under a different key
if 'crs' not in proj.keys():
    crs = proj['wkt']
else:
    crs = proj['crs']

transform = proj['transform']
scale = None

studyArea = composites.first().geometry()

# Decompress composites by dividing by 10000 for optical bands and add indices
composites = composites.select(['blue','green','red','nir','swir1','swir2'])\
    .map(lambda img: img.divide(10000).float().copyProperties(img,['system:time_start']))
composites = composites.map(getImagesLib.simpleAddIndices)\
                      .map(getImagesLib.getTasseledCap)\
                      .map(getImagesLib.simpleAddTCAngles)


print('Done')

EEException: Element.toDictionary: Parameter 'element' is required.

In [2]:
studyArea = (
    ee.FeatureCollection('projects/disco-glass-413820/assets/plot_min_bound_geom')
    .filter("site_id == 'UOFU_UTKA'")
    .first()
    .geometry()
)

In [16]:
import ee
import geemap

# Initialize Earth Engine
ee.Initialize()

# Define collection parameters
startYear = 2015
endYear = 2020
startDay = '01-01'
endDay = '12-31'
aoi = studyArea
index = 'NBR'
maskThese = ['cloud', 'shadow', 'snow', 'water']

# Define landtrendr parameters
runParams = {
  'maxSegments': 6,
  'spikeThreshold': 0.9,
  'vertexCountOvershoot': 3,
  'preventOneYearRecovery': True,
  'recoveryThreshold': 0.25,
  'pvalThreshold': 0.05,
  'bestModelProportion': 0.75,
  'minObservationsNeeded': 6
}

# Define change parameters
changeParams = {
  'delta': 'loss',
  'sort': 'greatest',
  'year': {'checked': True, 'start': 2015, 'end': 2020},
  'mag': {'checked': True, 'value': 200, 'operator': '>'},
  'dur': {'checked': True, 'value': 4, 'operator': '<'},
  'preval': {'checked': True, 'value': 300, 'operator': '>'},
  'mmu': {'checked': True, 'value': 11}
}

# Load the LandTrendr.js module
ltgee = geemap.js_snippet_to_py('users/emaprlab/public:Modules/LandTrendr.js')


# Add index to changeParams object
changeParams['index'] = index

# Run LandTrendr
lt = ltgee.runLT(startYear, endYear, startDay, endDay, aoi, index, [], runParams, maskThese)

# Get the change map layers
changeImg = ltgee.getChangeMap(lt, changeParams)

# Set visualization parameters
palette = ['#9400D3', '#4B0082', '#0000FF', '#00FF00', '#FFFF00', '#FF7F00', '#FF0000']
yodVizParms = {
  'min': startYear,
  'max': endYear,
  'palette': palette
}

magVizParms = {
  'min': 200,
  'max': 800,
  'palette': palette
}

# Display the change attribute map
Map = geemap.Map()
Map.centerObject(aoi, 11)
Map.addLayer(changeImg.select(['mag']), magVizParms, 'Magnitude of Change')
Map.addLayer(changeImg.select(['yod']), yodVizParms, 'Year of Detection')
Map

# Export change data to Google Drive
#region = aoi.buffer(100000).bounds()
#exportImg = changeImg.clip(region).unmask(0).short()
#geemap.ee_export_image_to_drive(exportImg, description='lt-gee_disturbance_map', folder='lt-gee_disturbance_map', 
#                                region=region, scale=30, crs='EPSG:5070')


AttributeError: 'NoneType' object has no attribute 'runLT'

In [18]:
import ee
import geemap

# Initialize Earth Engine
ee.Initialize()

# Define collection parameters
startYear = 1985
endYear = 2017
startDay = '06-20'
endDay = '09-20'
aoi = ee.Geometry.Point(-122.8848, 43.7929)
index = 'NBR'
maskThese = ['cloud', 'shadow', 'snow', 'water']

# Load the LandTrendr.js module
ltgee = geemap.js_snippet_to_py('users/emaprlab/public:Modules/LandTrendr.js')

# Define landtrendr parameters
runParams = {
  'maxSegments': 6,
  'spikeThreshold': 0.9,
  'vertexCountOvershoot': 3,
  'preventOneYearRecovery': True,
  'recoveryThreshold': 0.25,
  'pvalThreshold': 0.05,
  'bestModelProportion': 0.75,
  'minObservationsNeeded': 6
}

# Define change parameters
changeParams = {
  'delta': 'loss',
  'sort': 'greatest',
  'year': {'checked': True, 'start': 1986, 'end': 2017},
  'mag': {'checked': True, 'value': 200, 'operator': '>'},
  'dur': {'checked': True, 'value': 4, 'operator': '<'},
  'preval': {'checked': True, 'value': 300, 'operator': '>'},
  'mmu': {'checked': True, 'value': 11}
}

# Add index to changeParams object
changeParams['index'] = index

# Run LandTrendr
lt = ltgee.runLT(startYear, endYear, startDay, endDay, aoi, index, [], runParams, maskThese)

# Get the change map layers
changeImg = ltgee.getChangeMap(lt, changeParams)

# Set visualization parameters
palette = ['#9400D3', '#4B0082', '#0000FF', '#00FF00', '#FFFF00', '#FF7F00', '#FF0000']
yodVizParms = {
  'min': startYear,
  'max': endYear,
  'palette': palette
}

magVizParms = {
  'min': 200,
  'max': 800,
  'palette': palette
}

# Display the change attribute map
Map = geemap.Map()
Map.centerObject(aoi, 11)
Map.addLayer(changeImg.select(['mag']), magVizParms, 'Magnitude of Change')
Map.addLayer(changeImg.select(['yod']), yodVizParms, 'Year of Detection')
Map

# Export change data to Google Drive
region = aoi.buffer(100000).bounds()
exportImg = changeImg.clip(region).unmask(0).short()
geemap.ee_export_image_to_drive(exportImg, description='lt-gee_disturbance_map', folder='lt-gee_disturbance_map', 
                                region=region, scale=30, crs='EPSG:5070')


AttributeError: 'NoneType' object has no attribute 'runLT'

In [ ]:
import ee

'users/emaprlab/public':Modules/LandTrendr.js
m

In [ ]:
import ee

'users/emaprlab/public':Modules/LandTrendr.js
m

In [12]:
js_snippet = 'users/emaprlab/public:Modules/LandTrendr.js'

In [13]:
geemap.js_snippet_to_py(
    js_snippet, add_new_cell=True, import_ee=True, import_geemap=True, show_map=True
)


In [14]:
js_snippet

'users/emaprlab/public:Modules/LandTrendr.js'

In [ ]:
import ee
import geemap

m = geemap.Map()

'users/emaprlab/public':Modules/LandTrendr.js
m

In [11]:
# Load the LandTrendr.js module
#mymod = require('users/emaprlab/public:Modules/LandTrendr.js')

image = ee.Image('users/emaprlab/public:Modules/LandTrendr.js')
image
Map = geemap.Map()
Map.centerObject(image, 11)
Map.addLayer(changeImg.select(['mag']), magVizParms, 'Magnitude of Change')
Map.addLayer(changeImg.select(['yod']), yodVizParms, 'Year of Detection')
Map


EEException: Image.load: Image asset 'users/emaprlab/public:Modules/LandTrendr.js' not found (does not exist or caller does not have access).

In [23]:
composites = clipped_collection

NameError: name 'clipped_collection' is not defined

In [21]:
import ee
import geemap

# Initialize Earth Engine
ee.Initialize()

# Define collection parameters
startYear = 2015
endYear = 2020
aoi = studyArea

# Define a function to reduce an image collection to a single image
def reduceToSingeImageMockFunction(img):
    # Mock function to reduce image collection to a single image
    return img.mean()

# Initialize an empty image collection
srCollection = ee.ImageCollection([])

# Loop through each year from startYear to endYear
for year in range(startYear, endYear + 1):
    # Filter the Landsat image collection
    img = (ee.ImageCollection(composites)
           .filterBounds(aoi)
           .filterDate(str(year) + '-01-101', str(year) + '-12-31'))

    # Apply the reduce function to get a single image
    img = reduceToSingeImageMockFunction(img)

    # Select the band of interest
    tempCollection = img.select(['swir'])

    # Merge the current year's image with the collection
    if year == startYear:
        srCollection = tempCollection
    else:
        srCollection = srCollection.merge(tempCollection)

# Print the final image collection
print(srCollection.getInfo())


NameError: name 'composites' is not defined